Version 1.0.0

# Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise. 

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.

Now go through the notebook and complete the assignment.

In [2]:
import numpy as np
import pandas as pd 
import scipy.sparse

# Load the data

Let's load the test data. Note, that we don't have any training data here, just test data. Moreover, *we will not even use any features* of test objects. All we need to solve this task is the file with the indices for the pairs, that we need to compare.

Let's load the data with test indices.

In [3]:
test = pd.read_csv('../readonly/data_leakages_data/test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


For example, we can think that there is a test dataset of images, and each image is assigned a unique `Id` from $0$ to $N-1$ (N -- is the number of images). In the dataframe from above `FirstId` and `SecondId` point to these `Id`'s and define pairs, that we should compare: e.g. do both images in the pair belong to the same class or not. So, for example for the first row: if images with `Id=1427` and `Id=8053` belong to the same class, we should predict $1$, and $0$ otherwise. 

But in our case we don't really care about the images, and how exactly we compare the images (as long as comparator is binary).  

**We suggest you to try to solve the puzzle yourself first.** You need to submit a `.csv` file with columns `pairId` and `Prediction` to the grader. The number of submissions allowed is made pretty huge to let you explore the data without worries. The returned score should be very close to $1$.

**If you do not want to think much** -- scroll down and follow the instructions below.

In [145]:
unique_images = (test['FirstId'].append(test['SecondId'])).unique()
unique_images.shape[0]

26325

In [146]:
unique_images.min(), unique_images.max()

(0, 26324)

In [147]:
test.shape[0]

368550

In [148]:
test

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


# EDA and leakage intuition

As we already know, the key to discover data leakages is careful EDA. So let's start our work with some basic data exploration and build an intuition about the leakage.

First, check, how many different `id`s are there: concatenate `FirstId` and `SecondId` and print the number of unique elements. Also print minimum and maximum value for that vector.

In [150]:
unique_images = (test['FirstId'].append(test['SecondId']).unique())
unique_images.shape[0], unique_images.min(), unique_images.max()

(26325, 0, 26324)

and then print how many pairs we need to classify (it is basically the number of rows in the test set)

In [151]:
test.shape[0]

368550

Now print, how many distinct pairs it would be possible to create out of all "images" in the dataset?   

In [152]:
unique_images.shape[0]*(unique_images.shape[0]-1)

692979300

In [11]:
test.shape[0] / 2264950872.

0.0001627187611687853

So the number of pairs we are given to classify is very very small compared to the total number of pairs. 

To exploit the leak we need to **assume (or prove)**, that the total number of positive pairs is small, compared to the total number of pairs. For example: think about an image dataset with $1000$ classes, $N$ images per class. Then if the task was to tell whether a pair of images belongs to the same class or not, we would have $1000\frac{N(N-1)}{2}$ positive pairs, while total number of pairs was $\frac{1000N(1000N - 1)}{2}$.

Another example: in [Quora competitition](https://www.kaggle.com/c/quora-question-pairs) the task was to classify whether a pair of qustions are duplicates of each other or not. Of course, total number of question pairs is very huge, while number of duplicates (positive pairs) is much much smaller.

Finally, let's get a fraction of pairs of class `1`. We just need to submit a constant prediction "all ones" and check the returned accuracy. Create a dataframe with columns `pairId` and `Prediction`, fill it and export it to `.csv` file. Then submit to grader and examine grader's output. 

In [12]:
all_ones = pd.DataFrame({'pairId': range(test.shape[0]), 'Prediction': 1})
all_ones.reindex_axis(['pairId', 'Prediction'])

all_ones.to_csv('submission.csv', columns=['pairId', 'Prediction'], index=False)

So, we assumed the total number of pairs is much higher than the number of positive pairs, but it is not the case for the test set. It means that the test set is constructed not by sampling random pairs, but with a specific sampling algorithm. Pairs of class `1` are oversampled.

Now think, how we can exploit this fact? What is the leak here? If you get it now, you may try to get to the final answer yourself, othewise you can follow the instructions below.   

# Building a magic feature

In this section we will build a magic feature, that will solve the problem almost perfectly. The instructions will lead you to the correct solution, but please, try to explain the purpose of the steps we do to yourself -- it is very important.

## Incidence matrix

First, we need to build an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix). You can think of pairs `(FirstId, SecondId)` as of edges in an undirected graph. 

The incidence matrix is a matrix of size `(maxId + 1, maxId + 1)`, where each row (column) `i` corresponds `i-th` `Id`. In this matrix we put the value `1` to the position `[i, j]`, if and only if a pair `(i, j)` or `(j, i)` is present in  a given set of pais `(FirstId, SecondId)`. All the other elements in the incidence matrix are zeros.   

**Important!** The incidence matrices are typically very very sparse (small number of non-zero values). At the same time incidence matrices are usually huge in terms of total number of elements, and it is **impossible to store them in memory in dense format**. But due to their sparsity incidence matrices **can be easily represented as sparse matrices**. If you are not familiar with sparse matrices, please see [wiki](https://en.wikipedia.org/wiki/Sparse_matrix) and [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html). Please, use any of `scipy.sparse` constructors to build incidence matrix. 

For example, you can use this constructor: `scipy.sparse.coo_matrix((data, (i, j)))`. We highly recommend to learn to use different `scipy.sparse` constuctors, and matrices types, but if you feel you don't want to use them, you can always build this matrix with a simple `for` loop. You will need first to create a matrix using `scipy.sparse.coo_matrix((M, N), [dtype])` with an appropriate shape `(M, N)` and then iterate through `(FirstId, SecondId)` pairs and fill corresponding elements in matrix with ones. 

**Note**, that the matrix should be symmetric and consist only of zeros and ones. It is a way to check yourself.

In [17]:
from scipy.sparse import coo_matrix

mat_rows = test['FirstId']
mat_cols = test['SecondId']
inc_mat = coo_matrix((np.ones(len(mat_cols)), (mat_rows, mat_cols))) + coo_matrix((np.ones(len(mat_cols)), (mat_cols, mat_rows)))
inc_mat[inc_mat > 1] = 1

In [18]:
inc_mat

<26325x26325 sparse matrix of type '<class 'numpy.float64'>'
	with 736872 stored elements in Compressed Sparse Row format>

In [19]:
inc_mat.max()

1.0

In [20]:
# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872

It is convenient to have matrix in `csr` format eventually.

In [21]:
inc_mat = inc_mat.tocsr()

## Now build the magic feature

Why did we build the incidence matrix? We can think of the rows in this matix as of representations for the objects. `i-th` row is a representation for an object with `Id = i`. Then, to measure similarity between two objects we can measure similarity between their representations. And we will see, that such representations are very good.

Now select the rows from the incidence matrix, that correspond to `test.FirstId`'s, and `test.SecondId`'s.

In [92]:
# Note, scipy goes crazy if a matrix is indexed with pandas' series. 
# So do not forget to convert `pd.series` to `np.array`
# These lines should normally run very quickly 

rows_FirstId   = inc_mat[test['FirstId'].values]
rows_SecondId  = inc_mat[test['SecondId'].values]

Our magic feature will be the *dot product* between representations of a pair of objects. Dot product can be regarded as similarity measure -- for our non-negative representations the dot product is close to 0 when the representations are different, and is huge, when representations are similar. 

Now compute dot product between corresponding rows in `rows_FirstId` and `rows_SecondId` matrices.

In [93]:
f = np.sum(rows_FirstId.multiply(rows_SecondId),axis=1)

In [94]:
f.reshape((368550,)).shape

(1, 368550)

In [95]:
# Note, that in order to do pointwise multiplication in scipy.sparse you need to use function `multiply`
# regular `*` corresponds to matrix-matrix multiplication

f = np.sum(rows_FirstId.multiply(rows_SecondId),axis=1)
f = np.squeeze(np.asarray(f))

# Sanity check
assert f.shape == (368550, )

That is it! **We've built our magic feature.** 

# From magic feature to binary predictions

In [69]:
np.nonzero(rows_FirstId[0].toarray())

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([  541,  1171,  1209,  1371,  1987,  2934,  3467,  3512,  3651,
         3846,  4474,  4993,  5394,  5648,  7797,  8053,  9136,  9366,
        11235, 11488, 12360, 13829, 14291, 14546, 15819, 17301, 17491,
        17746, 19083, 20558, 20813, 22288, 22689, 22945, 24420, 24669]))

In [70]:
f[0]

20.0

In [71]:
np.nonzero(rows_SecondId[0].toarray())

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([  541,  1064,  1209,  1371,  1427,  1987,  2934,  3360,  3512,
         3651,  3846,  4474,  4886,  5287,  5648,  7690,  9033,  9366,
        11128, 11488, 12360, 13723, 14184, 14546, 15819, 17194, 17384,
        17746, 19083, 20453, 20813, 22183, 22582, 22945, 24314, 24669]))

In [72]:
test['FirstId'][0]

1427

In [73]:
test['SecondId'][0]

8053

In [88]:
pairs_1427 = test[(test['FirstId'] == 1427) | (test['SecondId'] == 1427)]
sorted((pairs_1427['FirstId'].append(pairs_1427['SecondId'])).unique())

[541,
 1171,
 1209,
 1371,
 1427,
 1987,
 2934,
 3467,
 3512,
 3651,
 3846,
 4474,
 4993,
 5394,
 5648,
 7797,
 8053,
 9136,
 9366,
 11235,
 11488,
 12360,
 13829,
 14291,
 14546,
 15819,
 17301,
 17491,
 17746,
 19083,
 20558,
 20813,
 22288,
 22689,
 22945,
 24420,
 24669]

In [81]:
pairs_1427

,pairId,FirstId,SecondId
0,0,1427,8053
9362,9362,1427,15819
26482,26482,1427,3846
35238,35238,1427,12360
52426,52426,1427,22945
60062,60062,13829,1427
82897,82897,1427,541
86120,86120,1427,24669
90175,90175,1427,17746
94207,94207,1427,4474


In [63]:
f.min()

14.0

But how do we convert this feature into binary predictions? We do not have a train set to learn a model, but we have a piece of information about test set: the baseline accuracy score that you got, when submitting constant. And we also have a very strong considerations about the data generative process, so probably we will be fine even without a training set. 

We may try to choose a thresold, and set the predictions to 1, if the feature value `f` is higer than the threshold, and 0 otherwise. What threshold would you choose? 

How do we find a right threshold? Let's first examine this feature: print frequencies (or counts) of each value in the feature `f`.

In [115]:
# For example use `np.unique` function, check for flags

print(sorted(list(zip(*np.unique(f, return_counts=True))), key=lambda x:x[1], reverse=True))

[(20.0, 183799), (14.0, 183279), (15.0, 852), (19.0, 546), (28.0, 54), (35.0, 14), (21.0, 6)]


In [97]:
np.unique(f)

array([ 14.,  15.,  19.,  20.,  21.,  28.,  35.])

Do you see how this feature clusters the pairs? Maybe you can guess a good threshold by looking at the values? 

In fact, in other situations it can be not that obvious, but in general to pick a threshold you only need to remember the score of your baseline submission and use this information. Do you understand why and how?  

Choose a threshold below: 

In [119]:
f[1]

14.0

In [120]:
rows_FirstId[1]

<1x26325 sparse matrix of type '<class 'numpy.float64'>'
	with 21 stored elements in Compressed Sparse Row format>

In [125]:
test['FirstId'][1]

17044

In [126]:
test['SecondId'][1]

7681

In [139]:
def get_paired_values(x):
    pairs = test[(test['FirstId'] == x) | (test['SecondId'] == x)]
    return sorted((pairs['FirstId'].append(pairs['SecondId'])).unique())

get_paired_values(17044)

[310,
 915,
 2507,
 3210,
 4738,
 6066,
 6872,
 7540,
 7681,
 8699,
 8884,
 10407,
 10979,
 12261,
 13577,
 14035,
 17044,
 18973,
 20305,
 22034,
 24165,
 25889]

In [140]:
f[0]

20.0

In [157]:
def show_f(idx):
    v = f[idx]
    first_id = test['FirstId'][idx]
    second_id = test['SecondId'][idx]
    pairs_first = get_paired_values(first_id)
    pairs_second = get_paired_values(second_id)
    print("v: {}, first: {} ({} neighbors), second: {} ({} neighbors)".format(v, first_id,  len(pairs_first),second_id, len(pairs_second)))
    print("first: {}".format(pairs_first))
    print("second: {}".format(pairs_second))

In [158]:
pred = f > 16

In [159]:
show_f(0)

v: 20.0, first: 1427 (37 neighbors), second: 8053 (37 neighbors)
first: [541, 1171, 1209, 1371, 1427, 1987, 2934, 3467, 3512, 3651, 3846, 4474, 4993, 5394, 5648, 7797, 8053, 9136, 9366, 11235, 11488, 12360, 13829, 14291, 14546, 15819, 17301, 17491, 17746, 19083, 20558, 20813, 22288, 22689, 22945, 24420, 24669]
second: [541, 1064, 1209, 1371, 1427, 1987, 2934, 3360, 3512, 3651, 3846, 4474, 4886, 5287, 5648, 7690, 8053, 9033, 9366, 11128, 11488, 12360, 13723, 14184, 14546, 15819, 17194, 17384, 17746, 19083, 20453, 20813, 22183, 22582, 22945, 24314, 24669]


In [160]:
show_f(1)

v: 14.0, first: 17044 (22 neighbors), second: 7681 (37 neighbors)
first: [310, 915, 2507, 3210, 4738, 6066, 6872, 7540, 7681, 8699, 8884, 10407, 10979, 12261, 13577, 14035, 17044, 18973, 20305, 22034, 24165, 25889]
second: [915, 1055, 3210, 3351, 4212, 4738, 4877, 5278, 6872, 6885, 7054, 7540, 7681, 8473, 8884, 9024, 10866, 10979, 11119, 11305, 11735, 13577, 13714, 14035, 14175, 17044, 17185, 17375, 18973, 20305, 20445, 22034, 22174, 22573, 24165, 24305, 25889]


In [161]:
show_f(2)

v: 20.0, first: 19237 (22 neighbors), second: 20966 (22 neighbors)
first: [1581, 2141, 2684, 3666, 5714, 5716, 5802, 8207, 8471, 9519, 9530, 11531, 11639, 11755, 12513, 14698, 15973, 17900, 19237, 20966, 23099, 24823]
second: [1581, 2141, 2684, 3666, 5714, 5716, 5802, 8207, 8471, 9519, 9530, 11531, 11639, 11755, 12513, 14698, 15973, 17900, 19237, 20966, 23099, 24823]


In [162]:
show_f(3)

v: 20.0, first: 8005 (37 neighbors), second: 20765 (22 neighbors)
first: [273, 991, 1379, 1940, 2408, 2528, 2568, 2773, 4096, 5201, 5601, 6232, 7113, 8005, 8470, 8638, 8939, 9318, 9949, 11442, 12065, 12312, 12941, 14498, 15128, 15771, 16404, 17698, 18331, 19667, 20765, 21397, 22496, 22897, 23525, 24622, 25252]
second: [991, 1379, 1940, 2408, 2528, 2773, 5201, 5601, 7113, 8005, 8470, 8939, 9318, 11442, 12312, 14498, 15771, 17698, 20765, 22496, 22897, 24622]


In [163]:
show_f(4)

v: 14.0, first: 16837 (22 neighbors), second: 599 (37 neighbors)
first: [599, 707, 3002, 4530, 6665, 7332, 7953, 8973, 10383, 10774, 10805, 11173, 12033, 12124, 13371, 15562, 16837, 18765, 20097, 21828, 23957, 25684]
second: [599, 707, 1826, 2894, 3002, 4422, 4530, 6558, 6665, 7224, 7332, 7914, 8209, 9024, 9352, 10275, 10383, 10668, 10774, 12004, 12129, 13264, 13371, 15454, 15562, 16729, 16837, 18657, 18765, 19991, 20097, 21721, 21828, 23849, 23957, 25577, 25684]


In [164]:
show_f(5)

v: 20.0, first: 3657 (37 neighbors), second: 12504 (22 neighbors)
first: [34, 196, 506, 926, 1572, 2132, 2331, 2580, 3022, 3657, 3857, 5005, 5793, 5993, 6264, 8198, 8399, 9510, 9710, 11630, 11828, 12504, 12705, 14690, 14890, 15964, 16165, 17891, 18092, 19228, 19428, 20957, 21158, 23090, 23288, 24814, 25013]
second: [196, 506, 926, 1572, 2132, 2580, 3022, 3657, 5005, 5793, 6264, 8198, 9510, 11630, 12504, 14690, 15964, 17891, 19228, 20957, 23090, 24814]


In [117]:
pred

array([ True, False,  True, ..., False, False, False], dtype=bool)

# Finally, let's create a submission

In [118]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

submission.to_csv('submission.csv', index=False)

Now submit it to the grader! It is not possible to submit directly from this notebook, as we need to submit a `csv` file, not a single number (limitation of Coursera platform). 

To download `submission.csv` file that you've just produced <a href='./submission.csv'>click here</a> (if the link opens in browser, right-click on it and shoose "Save link as"). Then go to [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.


If you did everything right, the score should be very high.

**Finally:** try to explain to yourself, why the whole thing worked out. In fact, there is no magic in this feature, and the idea to use rows in the incidence matrix can be intuitively justified.

# Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!